<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #2b6777; color:#ffffff;text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Airline passenger satisfaction (Part 2)</h1></div>

<center><a><img src="https://apartmentinteriors.ru/wp-content/uploads/samolot-charter-private-jet-charter-02.jpg" border="3" width=800 height=600 class="center"></a>

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Introduction</h1></div>

<b>In the first part:</b>
<ul>
    <li>This dataset was described and analyzed in detail.</li>
    <li>Some data preprocessing was done.</li>
    <li>Several machine learning algorithms (kNN, SVC, Random Forest, Extremely Randomized Trees, AdaBoost and GBT) were launched.
    <li>The results of their work were presented.</li>
</ul>

<b>In this part:</b>
<ul>
    <li>The neural network will be trained.</li>
    <li>The hyperparameters will be selected in several ways.</li>
    <li>Conclusions based on the results of the work are given.</li>
</ul>

***
<b>If you haven't seen the first part, I recommend that you first read it.</b>

Link to the first part: [**Airline Passenger Satisfaction (Part 1)**](https://www.kaggle.com/code/frixinglife/airline-passenger-satisfaction-part-1)
***


<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Task description</h1></div>

**There is the following information about the passengers of some airline:**

1. **Gender:** male or female
2. **Customer type:** regular or non-regular airline customer
3. **Age:** the actual age of the passenger
4. **Type of travel:** the purpose of the passenger's flight (personal or business travel)
5. **Class:** business, economy, economy plus
6. **Flight distance**
7. **Inflight wifi service:** satisfaction level with Wi-Fi service on board (0: not rated; 1-5)
8. **Departure/Arrival time convenient:** departure/arrival time satisfaction level (0: not rated; 1-5)
9. **Ease of Online booking:** online booking satisfaction rate (0: not rated; 1-5)
10. **Gate location:** level of satisfaction with the gate location (0: not rated; 1-5)
11. **Food and drink:** food and drink satisfaction level (0: not rated; 1-5)
12. **Online boarding:** satisfaction level with online boarding (0: not rated; 1-5)
13. **Seat comfort:** seat satisfaction level (0: not rated; 1-5)
14. **Inflight entertainment:** satisfaction with inflight entertainment (0: not rated; 1-5)
15. **On-board service:** level of satisfaction with on-board service (0: not rated; 1-5)
16. **Leg room service**: level of satisfaction with leg room service (0: not rated; 1-5)
17. **Baggage handling:** level of satisfaction with baggage handling (0: not rated; 1-5)
18. **Checkin service:** level of satisfaction with checkin service (0: not rated; 1-5)
19. **Inflight service:** level of satisfaction with inflight service (0: not rated; 1-5)
20. **Cleanliness:** level of satisfaction with cleanliness (0: not rated; 1-5)
21. **Departure delay in minutes**
22. **Arrival delay in minutes**

This data set contains a survey on <b>air passenger satisfaction</b>. The following <b>classification problem</b> is set:

It is necessary to predict which of the <b>two</b> levels of satisfaction with the airline the passenger belongs to:
<ol>
    <li><em>Satisfaction</em></li>
    <li><em>Neutral or dissatisfied</em></li>
</ol>

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Reading data</h1></div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
data = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/train.csv")

Table dimensions:

In [ ]:
data.shape

Each row corresponds to one passenger, and each column to a specific feature.<br>
Let's look at the first row of the data set:

In [ ]:
data.head()

Let's take a closer look at the dataset data:

In [ ]:
data.info()

You may notice the following:
<ol>
     <li><b>The column</b> corresponding to the <b>Arrival Delay in Minutes feature has 310 missing values</b>.</li>
     <li><b>The first two features are useless and will not affect the classification</b>, so you should get rid of them.</li>
     <li><b>Many columns contain categorical values</b> but are of type 'object' or 'int64'. Let's replace this type with a special one designed for storing categorical values.</li>
</ol>

In [ ]:
data = data.drop(data.iloc[:,[0, 1]], axis = 1)

In [ ]:
categorical_indexes = [0, 1, 3, 4] + list(range(6, 20))
data.iloc[:,categorical_indexes] = data.iloc[:,categorical_indexes].astype('category')

Now the dataset information looks like this:

In [ ]:
data.info()

The first 22 features have been detailed above. The <b>satisfaction</b> feature is the target.

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Filling in missing values</h1></div>

Let's see how many missing values are in each column of the table:

In [ ]:
data.isna().sum()

Fill in the missing values with <b>medians</b> in the columns corresponding to quantitative features:

In [ ]:
data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].median(axis = 0), inplace = True)

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

This table shows that there are no more missing values.

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Handling categorical features</h1></div>

We divide the signs into quantitative and categorical:

In [ ]:
numerical_columns = [c for c in data.columns if data[c].dtype.name != 'category']
numerical_columns.remove('satisfaction')
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'category']
data_describe = data.describe(include = ['category'])

We divide categorical features into binary and non-binary:

In [ ]:
binary_columns = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print(binary_columns, nonbinary_columns)

Let's look at the unique values for each binary feature:

In [ ]:
for col in binary_columns:
    print(col, ': ', end = '')
    for uniq in data[col].unique():
        if uniq == data[col].unique()[-1]:
            print(uniq, end = '.')
        else:
            print(uniq, end = ', ')
    print()

Let's do the binarization:

In [ ]:
for col in binary_columns:
    data[col] = data[col].astype('object')
    k = 0
    for uniq in data[col].unique():
        data.at[data[col] == uniq, col] = k
        k +=1 
for col in binary_columns:
    print(data[col].describe(), end = '\n\n')

Now let's look at non-binary categorical features:

In [ ]:
data[nonbinary_columns]

The following vectorization method is applicable to non-binary features:

The feature j, which takes s values, will be replaced by s features, which take the values 0 or 1, depending on what the value of the original feature j is.

This vectorization is carried out by the get_dummies method:

In [ ]:
data_nonbinary = pd.get_dummies(data[nonbinary_columns])
print(data_nonbinary.columns)

In [ ]:
len(data_nonbinary.columns)

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Normalization of quantitative features</h1></div>

We have the following quantitative characteristics:

In [ ]:
data_numerical = data[numerical_columns]
data_numerical.describe()

Let's perform a <b>standardization</b> (<em>linear transformation that reduces all values to zero mean and one standard deviation</em>) of all quantitative features:

In [ ]:
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)

In [ ]:
data_numerical.describe()

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Table formation</h1></div>

We join all the transformed columns into one table:

In [ ]:
target = data['satisfaction']
data = pd.concat((data_numerical, data_nonbinary, data[binary_columns]), axis = 1)
print(data.shape)

Now it looks like this:

In [ ]:
data.describe()

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Neural network</h1></div>

### Splitting the data on training and test sets

Get <b>X</b> and <b>y</b>:

In [ ]:
X = data
y = target
N, d = X.shape
N, d

In [ ]:
X.columns

In [ ]:
y

Training a neural network takes a lot of time, so let's take not all the data, but only part of it:

In [ ]:
N_train = 27000
N_test = 3000

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = N_train, test_size = N_test, 
                                                    stratify = y, random_state = 42)

X_train.shape, X_test.shape 

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Selection of hyperparameters</h1></div>

The selection of hyperparameters will occur in <b>two ways</b>.

<b>First way:</b>
<center><a><img src="https://raw.githubusercontent.com/Frixinglife/Homework_ML/60e1d5847d6f7c728025caa39f350074f58b299d/Kaggle.svg" border="0" class="center">
</a>

<b>Second way:</b>

The second way is that the following hyperparameters will be iterated simultaneously (first the first two, and then all three at once):

<ul>
    <li>Alpha parameter</li>
    <li>Number of neurons in the first hidden layer</li>
    <li>Number of neurons in the second hidden layer</li>
</ul>

But we will enumerate them in some neighborhood of the parameters obtained earlier, otherwise the enumeration will be very long.

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>The first way to select hyperparameters</h1></div>

### Selection of the optimal regularization parameter alpha (for 1 hidden layer of 50 neurons)

In [ ]:
%%time
alpha_arr = np.logspace(-3, 2, 21)
test_err = []
train_err = []
train_acc = []
test_acc = []

for alpha in alpha_arr:
    mlp_model = MLPClassifier(alpha = alpha, 
                              hidden_layer_sizes = 50,
                              solver = 'lbfgs', 
                              max_iter = 1000, 
                              activation = 'logistic',
                              random_state = 42)
    
    
    mlp_model.fit(X_train, y_train)

    y_train_pred = mlp_model.predict(X_train)
    y_test_pred = mlp_model.predict(X_test)
    
    train_err.append(np.mean(y_train != y_train_pred))
    test_err.append(np.mean(y_test != y_test_pred))
    train_acc.append(accuracy_score(y_train, y_train_pred))
    test_acc.append(accuracy_score(y_test, y_test_pred))

In [ ]:
plt.semilogx(alpha_arr, train_err, 'b-o', label = 'train')
plt.semilogx(alpha_arr, test_err, 'r-o', label = 'test')
plt.xlim([np.min(alpha_arr), np.max(alpha_arr)])
plt.title('Error vs. alpha')
plt.xlabel('Alpha')
plt.ylabel('Error')
plt.legend()
pass

In [ ]:
plt.semilogx(alpha_arr, train_acc, 'r-o', label = 'train')
plt.semilogx(alpha_arr, test_acc, 'b-o', label = 'test')
plt.xlim([np.min(alpha_arr), np.max(alpha_arr)])
plt.title('Accuracy vs. alpha')
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.legend()
pass

In [ ]:
min_train_err = np.min(train_err)
min_test_err = np.min(test_err)
print('Minimum error on the training set: ', min_train_err)
print('Minimum error on the testing set: ', min_test_err)

In [ ]:
alpha_local_opt = alpha_arr[test_err == min_test_err][0]
print('Optimal alpha value: ', alpha_local_opt)

### Classifier training at the optimal alpha value (for 1 hidden layer of 50 neurons)

In [ ]:
%%time
mlp_model = MLPClassifier(alpha = alpha_local_opt, 
                          hidden_layer_sizes = 50,
                          solver = 'lbfgs', 
                          max_iter = 1000, 
                          activation = 'logistic',
                          random_state = 42)

mlp_model.fit(X_train, y_train)

In [ ]:
y_train_pred = mlp_model.predict(X_train)
y_test_pred = mlp_model.predict(X_test)

err_train = np.mean(y_train != y_train_pred)
err_test = np.mean(y_test != y_test_pred)

print('Error on the training set: ', err_train)
print('Error on the testing set: ', err_test)

### Selection of the neurons number of 1 hidden layer for the optimal (for 1 hidden layer of 50 neurons) alpha value

In [ ]:
%%time
hidden_layer = np.arange(1, 51)
test_err = []
train_err = []
train_acc = []
test_acc = []

for size in hidden_layer:
    mlp_model = MLPClassifier(alpha = alpha_local_opt, 
                              hidden_layer_sizes = size,
                              solver = 'lbfgs', 
                              max_iter = 1000, 
                              activation = 'logistic',
                              random_state = 42)
    
    
    mlp_model.fit(X_train, y_train)

    y_train_pred = mlp_model.predict(X_train)
    y_test_pred = mlp_model.predict(X_test)
    
    train_err.append(np.mean(y_train != y_train_pred))
    test_err.append(np.mean(y_test != y_test_pred))
    train_acc.append(accuracy_score(y_train, y_train_pred))
    test_acc.append(accuracy_score(y_test, y_test_pred))

In [ ]:
plt.semilogx(hidden_layer, train_err, 'b-o', label = 'train')
plt.semilogx(hidden_layer, test_err, 'r-o', label = 'test')
plt.xlim([np.min(hidden_layer), np.max(hidden_layer)])
plt.title('Error vs. number of hidden neurons')
plt.xlabel('Number of hidden neurons')
plt.ylabel('Error')
plt.legend()
pass

In [ ]:
plt.semilogx(hidden_layer, train_acc, 'b-o', label = 'train')
plt.semilogx(hidden_layer, test_acc, 'r-o', label = 'test')
plt.xlim([np.min(hidden_layer), np.max(hidden_layer)])
plt.title('Accuracy vs. number of hidden neurons')
plt.xlabel('Number of hidden neurons')
plt.ylabel('Accuracy')
plt.legend()
pass

In [ ]:
min_train_err = np.min(train_err)
min_test_err = np.min(test_err)
print('Minimum error on the training set: ', min_train_err)
print('Minimum error on the testing set: ', min_test_err)

In [ ]:
hidden_layer_local_opt_1 = hidden_layer[test_err == min_test_err][0]
print('The optimal value of the neurons number in the hidden layer: ', hidden_layer_local_opt_1)

### Training a neural network with the optimal alpha value (for 1 hidden layer of 50 neurons) and the optimal neurons number in the first hidden layer (for a given alpha)

In [ ]:
%%time
mlp_model = MLPClassifier(alpha = alpha_local_opt, 
                          hidden_layer_sizes = hidden_layer_local_opt_1,
                          solver = 'lbfgs', 
                          max_iter = 1000, 
                          activation = 'logistic',
                          random_state = 42)

mlp_model.fit(X_train, y_train)

In [ ]:
y_train_pred = mlp_model.predict(X_train)
y_test_pred = mlp_model.predict(X_test)

err_train = np.mean(y_train != y_train_pred)
err_test = np.mean(y_test != y_test_pred)

print('Error on the training set: ', err_train)
print('Error on the testing set: ', err_test)

### Selection of the neurons number in the second hidden layer with the optimal alpha value (for 1 hidden layer of 50 neurons) and the optimal neurons number in the first hidden layer (for a given alpha)

In [ ]:
%%time
hidden_layer = np.arange(1, 51)
test_err = []
train_err = []
train_acc = []
test_acc = []

for size in hidden_layer:
    mlp_model = MLPClassifier(alpha = alpha_local_opt, 
                              hidden_layer_sizes = (hidden_layer_local_opt_1, size),
                              solver = 'lbfgs', 
                              max_iter = 1000, 
                              activation = 'logistic',
                              random_state = 42)
    
    mlp_model.fit(X_train, y_train)

    y_train_pred = mlp_model.predict(X_train)
    y_test_pred = mlp_model.predict(X_test)
    
    train_err.append(np.mean(y_train != y_train_pred))
    test_err.append(np.mean(y_test != y_test_pred))
    train_acc.append(accuracy_score(y_train, y_train_pred))
    test_acc.append(accuracy_score(y_test, y_test_pred))

In [ ]:
plt.semilogx(hidden_layer, train_err, 'b-o', label = 'train')
plt.semilogx(hidden_layer, test_err, 'r-o', label = 'test')
plt.xlim([np.min(hidden_layer), np.max(hidden_layer)])
plt.title('Error vs. number of hidden neurons (second layer)')
plt.xlabel('Number of hidden neurons (second layer)')
plt.ylabel('Error')
plt.legend()
pass

In [ ]:
plt.semilogx(hidden_layer, train_acc, 'b-o', label = 'train')
plt.semilogx(hidden_layer, test_acc, 'r-o', label = 'test')
plt.xlim([np.min(hidden_layer), np.max(hidden_layer)])
plt.title('Accuracy vs. number of hidden neurons (second layer)')
plt.xlabel('Number of hidden neurons (second layer)')
plt.ylabel('Accuracy')
plt.legend()
pass

In [ ]:
min_train_err = np.min(train_err)
min_test_err = np.min(test_err)
print('Minimum error on the training set: ', min_train_err)
print('Minimum error on the testing set: ', min_test_err)

In [ ]:
hidden_layer_local_opt_2 = hidden_layer[test_err == min_test_err][0]
print('The optimal value of the number of neurons in the second hidden layer: ', hidden_layer_local_opt_2)

### Training a neural network with the optimal alpha value (for 1 hidden layer of 50 neurons), optimal neurons numbers in the first and second hidden layers

In [ ]:
%%time
mlp_model = MLPClassifier(alpha = alpha_local_opt, 
                          hidden_layer_sizes = (hidden_layer_local_opt_1, hidden_layer_local_opt_2),
                          solver = 'lbfgs', 
                          max_iter = 1000, 
                          activation = 'logistic',
                          random_state = 42)

mlp_model.fit(X_train, y_train)

In [ ]:
y_train_pred = mlp_model.predict(X_train)
y_test_pred = mlp_model.predict(X_test)

err_train = np.mean(y_train != y_train_pred)
err_test = np.mean(y_test != y_test_pred)

print('Error on the training set: ', err_train)
print('Error on the testing set: ', err_test)

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>The second way to select hyperparameters</h1></div>

### Simultaneous enumeration of alpha and the neurons number in the hidden layer (for a neural network with one hidden layer)

In [ ]:
%%time
hidden_layer = np.arange(hidden_layer_local_opt_1 - 2, hidden_layer_local_opt_1 + 2)
alpha_arr = np.logspace(-3, 2, 21)
test_err = []
train_err = []
train_acc = []
test_acc = []

opt_params = []

for size in hidden_layer:
    for alpha in alpha_arr:
        mlp_model = MLPClassifier(alpha = alpha, 
                                  hidden_layer_sizes = size,
                                  solver = 'lbfgs', 
                                  max_iter = 1000, 
                                  activation = 'logistic',
                                  random_state = 42)


        mlp_model.fit(X_train, y_train)

        y_train_pred = mlp_model.predict(X_train)
        y_test_pred = mlp_model.predict(X_test)

        train_err.append(np.mean(y_train != y_train_pred))
        test_err.append(np.mean(y_test != y_test_pred))
        train_acc.append(accuracy_score(y_train, y_train_pred))
        test_acc.append(accuracy_score(y_test, y_test_pred))
        
        params = {'alpha': alpha, 'hidden_layer': size} 
        opt_params.append(params)

In [ ]:
min_train_err = np.min(train_err)
min_test_err = np.min(test_err)
print('Minimum error on the training set: ', min_train_err)
print('Minimum error on the testing set: ', min_test_err)

In [ ]:
index_opt = 0
for index in range(len(test_err)):
    if test_err[index] == min_test_err:
        index_opt = index
        break
        
hidden_layer_opt = opt_params[index_opt]['hidden_layer']
print('The optimal value of the neurons number in the hidden layer: ', hidden_layer_opt)

alpha_opt = opt_params[index_opt]['alpha']
print('Optimal alpha value: ', alpha_opt)

### Simultaneous enumeration of alpha, the neurons number in the first and second hidden layers

In [ ]:
%%time
hidden_layer_first = np.arange(hidden_layer_local_opt_1 - 2, hidden_layer_local_opt_1 + 2)
hidden_layer_second = np.arange(hidden_layer_local_opt_2 - 2, hidden_layer_local_opt_2 + 2)
alpha_arr = np.logspace(-3, 2, 21)
test_err = []
train_err = []
train_acc = []
test_acc = []

opt_params = []

for size_first in hidden_layer_first:
    for size_second in hidden_layer_second:
        for alpha in alpha_arr:
            mlp_model = MLPClassifier(alpha = alpha, 
                                      hidden_layer_sizes = (size_first, size_second), 
                                      solver = 'lbfgs', 
                                      max_iter = 1000, 
                                      activation = 'logistic',
                                      random_state = 42)


            mlp_model.fit(X_train, y_train)

            y_train_pred = mlp_model.predict(X_train)
            y_test_pred = mlp_model.predict(X_test)

            train_err.append(np.mean(y_train != y_train_pred))
            test_err.append(np.mean(y_test != y_test_pred))
            train_acc.append(accuracy_score(y_train, y_train_pred))
            test_acc.append(accuracy_score(y_test, y_test_pred))
            
            params = {'alpha': alpha, 'hidden_layer_first': size_first, 'hidden_layer_second': size_second} 
            opt_params.append(params)

In [ ]:
min_train_err = np.min(train_err)
min_test_err = np.min(test_err)
print('Minimum error on the training set: ', min_train_err)
print('Minimum error on the testing set: ', min_test_err)

In [ ]:
index_opt = 0
for index in range(len(test_err)):
    if test_err[index] == min_test_err:
        index_opt = index
        break
        
hidden_layer_opt_1 = opt_params[index_opt]['hidden_layer_first']
print('The optimal value of the neurons number in the first hidden layer: ', hidden_layer_opt_1)

hidden_layer_opt_2 = opt_params[index_opt]['hidden_layer_second']
print('The optimal value of the neurons number in the second hidden layer: ', hidden_layer_opt_2)

alpha_opt = opt_params[index_opt]['alpha']
print('Optimal alpha value: ', alpha_opt)

***
<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Conclusions</h1></div>

> <ul>
> <li>The error on the test sample is lesser than for other classifiers (in part 1).</li>
> <li>The training didn't take place on the entire sample, but only on part of it. Despite this a very good result was achieved.</li>
> <li>Since the quality of training of neural networks depends on the amount of input data, with an increase in the training sample, a result that is better than the previous one can be achieved.</li>
> <li>Hyperparameters were selected in narrow ranges. By increasing the ranges, a smaller error can be achieved.</li>
> </ul>

***